In [0]:
import torch
import os

In [0]:
# GPU Check

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())
print(torch.cuda.get_device_name(device))

Available devices  1
Current cuda device  0
Tesla P100-PCIE-16GB


In [0]:
# TPU Check

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

TPU address is grpc://10.11.126.82:8470


In [0]:
import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe
import random

# Approach 1:
# set up fields
# TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
# LABEL = data.Field(sequential=False)

SEED = 1234 # Random Seed for reproductivity

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

In [0]:
train, test = datasets.IMDB.splits(TEXT, LABEL)

print(f'Number of training examples: {len(train)}')
print(f'Number of testing examples: {len(test)}')

print(vars(train.examples[0]))

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.3MB/s]


Number of training examples: 25000
Number of testing examples: 25000
{'text': ['Please', 'see', 'also', 'my', 'comment', 'on', 'Die', 'Nibelungen', 'part', '1', ':', 'Siegfried.<br', '/><br', '/>The', 'second', 'part', 'of', 'UFA', 'studio', "'s", 'gargantuan', 'production', 'of', 'the', 'Nibelungen', 'saga', 'continues', 'in', 'the', 'stylised', ',', 'symphonic', 'and', 'emotionally', 'detached', 'manner', 'of', 'its', 'predecessor', '.', 'However', ',', 'whereas', 'part', 'one', 'was', 'a', 'passionless', 'portrayal', 'of', 'individual', 'acts', 'of', 'heroism', ',', 'part', 'two', 'is', 'a', 'chaotic', 'depiction', 'of', 'bloodletting', 'on', 'a', 'grand', 'scale.<br', '/><br', '/>As', 'in', 'part', 'one', ',', 'director', 'Fritz', 'Lang', 'maintains', 'a', 'continuous', 'dynamic', 'rhythm', ',', 'with', 'the', 'pace', 'of', 'the', 'action', 'and', 'the', 'complexity', 'of', 'the', 'shot', 'composition', 'rising', 'and', 'falling', 'smoothly', 'as', 'the', 'tone', 'of', 'each', 'sce

In [0]:
# make splits for data
# by default this splits 70:30

train, valid = train.split(random_state = random.seed(SEED))

In [0]:
print('train.fields', train.fields)
print(f'Number of training examples: {len(train)}')
print(f'Number of validation examples: {len(valid)}') 
print(f'Number of testing examples: {len(test)}')

train.fields {'text': <torchtext.data.field.Field object at 0x7f54ca698c18>, 'label': <torchtext.data.field.LabelField object at 0x7f54ca698c50>}
Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [0]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train)

In [0]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [0]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 201852), (',', 192394), ('.', 164699), ('and', 109095), ('a', 108947), ('of', 100771), ('to', 93575), ('is', 75983), ('in', 61025), ('I', 54169), ('it', 53440), ('that', 49044), ('"', 44304), ("'s", 43028), ('this', 42340), ('-', 37076), ('/><br', 35552), ('was', 35060), ('as', 30419), ('with', 29688)]


In [0]:
print(TEXT.vocab.itos[:10])
print(LABEL.vocab.stoi)

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']
defaultdict(<function _default_unk_index at 0x7f54c2a851e0>, {'neg': 0, 'pos': 1})


In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size = BATCH_SIZE,
    device = device)

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [0]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [0]:
criterion = nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/lottery-ticket-hypothesis-for-text-classification/')

In [0]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


Epoch: 01 | Epoch Time: 0m 15s
	Train Loss: 0.694 | Train Acc: 50.12%
	 Val. Loss: 0.697 |  Val. Acc: 49.95%
Epoch: 02 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 49.93%
	 Val. Loss: 0.697 |  Val. Acc: 49.93%
Epoch: 03 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 50.17%
	 Val. Loss: 0.697 |  Val. Acc: 50.86%
Epoch: 04 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 49.59%
	 Val. Loss: 0.697 |  Val. Acc: 49.50%
Epoch: 05 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 50.03%
	 Val. Loss: 0.697 |  Val. Acc: 49.92%


In [0]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.710 | Test Acc: 47.39%
